# Data Preprocessing

In [1]:
#TODO authors regex regular expression
#TODO  dates


### lowerCase

In [2]:
def toLower(text):
    return text.lower()

### Numbers to words

In [3]:
import inflect
p = inflect.engine()

def isFLoat(strNum):
    try:
        float(strNum)
        return True
    except:
        return False


def converteNumbers(text):
    tempText = text.split()
    newText = []
    for word in tempText:
        if word.isdigit() or isFLoat(word):
            temp = p.number_to_words(word)
            newText.append(temp)
        else:
            newText.append(word)
    tempText = ' '.join(newText)
    return tempText

### remove punctuations

In [4]:
import string
translator = str.maketrans('','',string.punctuation)
def removePunctuation(text):
    global translator
    return text.translate(translator)

### remove WhiteSpaces

In [5]:
def removeWhiteSpace(text):
    return " ".join(text.split())

### remove Stop Words

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def removeStopWords(text):
    sw = set(stopwords.words("english"))
    wt = word_tokenize(text)
    filteredText = [word for word in wt if word not in sw]
    return ' '.join(filteredText)

### Stemming

In [7]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

def stemWords(text):
    global stemmer
    wt = word_tokenize(text)
    stems = [stemmer.stem(word) for word in wt]
    return ' '.join(stems)

### lemmatization

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatizeWords(text):
    wt = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(word, pos='v') for word in wt]
    return ' '.join(lemmas)

### data preprocessing

In [9]:
import pandas as pd
import re

caRegex = r'CA[0-9]* ?[A-Z]{2} '
def TitlePreProcesse(t):
    tempText = toLower(t)
    tempText = converteNumbers(tempText)
    tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def abstractPreProcesse(a):
    tempText = toLower(a)
    tempText = converteNumbers(tempText)
    tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def publicationPreProcesse(p):
    # tempText = toLower(p)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    # tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)

    # TODO remove cacm word and coverte the rest of the data to dateType 
    tempText = p.replace('CACM ','')
    return pd.to_datetime(tempText)

def authorPreProcesse(a):
    tempText = toLower(a)
    # tempText = converteNumbers(tempText)
    
    # TODO take the first wordbefore ,
    lis = tempText.split(' ')
    names = ' '
    l = []
    for word in lis:
      if ',' in word:
          l.append(removePunctuation(word.replace(',','')))
    names = ' '.join(l)

    # tempText = removePunctuation(tempText)
    
    # tempText = removeWhiteSpace(lis[0])
    
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return names

def kPreProcesse(k):
    tempText = toLower(k)
    tempText = converteNumbers(tempText)
    tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(tempText)
    tempText = removeStopWords(tempText)
    tempText = stemWords(tempText)
    tempText = lemmatizeWords(tempText)
    return tempText

def cPreProcesse(c):
    # tempText = toLower(c)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

def infoPreProcesse(i):
    # tempText = toLower(i)
    # tempText = converteNumbers(tempText)
    # tempText = removePunctuation(tempText)
    #TODO remove the (CA581202 JB) ... and converte the rest to date type
    tempText = re.sub(caRegex, '', i)
    tempText = pd.to_datetime(tempText)
    #tempText = removeWhiteSpace(tempText)
    # tempText = removeStopWords(tempText)
    # tempText = stemWords(tempText)
    # tempText = lemmatizeWords(tempText)
    return tempText

In [10]:
import pandas as pd
def preprocessedData(dataFrame:pd.DataFrame):
    pdataFrame = pd.DataFrame(columns=['.I','data','.B','.C','.N','.X'])
    seriesDict:dict = {'.I':None, 'data':None, '.B':None, '.C':None,'.N':None, '.X':None} 
    seriesData = seriesDict.copy()
    for i in dataFrame.index:
        try:
            templist = []
            tempC = tempB = tempI = None
            if not dataFrame.loc[i, '.T'] == '':
                templist.append(TitlePreProcesse(dataFrame.loc[i, '.T']))
            if not dataFrame.loc[i, '.W'] == '':
                templist.append(abstractPreProcesse(dataFrame.loc[i, '.W']))
            if not dataFrame.loc[i, '.B'] == '':
                tempB = publicationPreProcesse(dataFrame.loc[i, '.B'])
            if not dataFrame.loc[i, '.A'] == '':
                templist.append(authorPreProcesse(dataFrame.loc[i, '.A']))
            if not dataFrame.loc[i, '.K'] == '':
                templist.append(kPreProcesse(dataFrame.loc[i, '.K']))
            if not dataFrame.loc[i, '.C'] == '':
                tempC = cPreProcesse(dataFrame.loc[i, '.C'])
            if not dataFrame.loc[i, '.N'] == '':
                tempI = infoPreProcesse(dataFrame.loc[i, '.N'])

            seriesData['.I'] = i
            seriesData['data'] = ' '.join(templist)
            seriesData['.B'] = tempB
            seriesData['.C'] = tempC
            seriesData['.N'] = tempI
            seriesData['.X'] = dataFrame.loc[i,'.X']
            pdataFrame = pdataFrame.append(seriesData, ignore_index=True)
        except:
            print(i)
            raise 
    
    pdataFrame.set_index('.I', inplace=True)
    return pdataFrame


In [11]:
import pandas as pd
data = pd.read_csv('../../cacmData/cacmDataCleanedV1.csv')
data.set_index('.I', inplace=True)
data.fillna('', inplace=True)
data.head()

,.T,.W,.B,.A,.K,.C,.N,.X
.I,,,,,,,,
1,Preliminary Report-International Algebraic Lan...,,"CACM December, 1958","Perlis, A. J. Samelson, K.",,,CA581203 JB March 22 1978 8:28 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
2,Extraction of Roots by Repeated Subtractions f...,,"CACM December, 1958","Sugai, I.",,,CA581202 JB March 22 1978 8:29 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
3,Techniques Department on Matrix Program Schemes,,"CACM December, 1958","Friedman, M. D.",,,CA581201 JB March 22 1978 8:30 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
4,Glossary of Computer Engineering and Programmi...,,"CACM November, 1958",,,,CA581103 JB March 22 1978 8:32 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."
5,Two Square-Root Approximations,,"CACM November, 1958","Wadey, W. G.",,,CA581102 JB March 22 1978 8:33 PM,"100,5,1-123,5,1-164,5,1-1,5,1-1,5,1-1,5,1-205,..."


In [12]:
d = preprocessedData(data)
d.set_index('.I', inplace=True)
data.fillna('', inplace=True)
data.head()

908


ParserError: Unknown string format: CA630312a JB March 14  1978  1:38 PM

In [ ]:
d[-5:]

In [ ]:
data[906:909]